In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, expr, coalesce, current_date, year, udf
from pyspark.ml.linalg import Vectors, VectorUDT
from sentence_transformers import SentenceTransformer
from pyspark.sql.types import ArrayType, StringType




# Create a Spark session
spark = SparkSession.builder.appName("BookEmbeddings1").getOrCreate()

# Read data directly into DataFrame with appropriate CSV options
csv_path = "books_task.csv"
df = spark.read.option("header", "true").option("quote", "\"").option("escape", "\"").csv(csv_path)

df.printSchema()

# Rename the '_c0' column to 'Index'
df = df.withColumnRenamed("_c0", "Index")

# Convert 'publishedDate' to date format
df = df.withColumn("publishedDate", col("publishedDate").cast("date"))

# Count missing values in each column
missing_counts = [df.where(col(c).isNull()).count() for c in df.columns]

# Create a dictionary to map column names to missing value counts
missing_dict = dict(zip(df.columns, missing_counts))

# Display the missing value counts for each column
for column, missing_count in missing_dict.items():
    print(f"Column '{column}': {missing_count} missing values")
    
# Replace missing values with a default date 
default_date = "1900-01-01"
df = df.withColumn("publishedDate", when(col("publishedDate").isNull(), default_date).otherwise(col("publishedDate")))

df.printSchema()

# Count missing values in each column
missing_counts = [df.where(col(c).isNull()).count() for c in df.columns]

# Create a dictionary to map column names to missing value counts
missing_dict = dict(zip(df.columns, missing_counts))

# Display the missing value counts for each column
for column, missing_count in missing_dict.items():
    print(f"Column '{column}': {missing_count} missing values")
    

# Define a UDF to convert string representation of list to actual list
@udf(ArrayType(StringType()))
def convert_to_list(authors_str):
    return (authors_str[1:-1].replace("'", "").split(',')) if authors_str is not None else []

# Apply the UDF to the 'authors' column
df = df.withColumn("authors_list", convert_to_list(col("authors")))

# Apply the UDF to the 'categories' column
df = df.withColumn("categories_list", convert_to_list(col("categories")))


#Count missing values in each column
missing_counts = [df.where(col(c).isNull()).count() for c in df.columns]

# Create a dictionary to map column names to missing value counts
missing_dict = dict(zip(df.columns, missing_counts))

# Display the missing value counts for each column
for column, missing_count in missing_dict.items():
    print(f"Column '{column}': {missing_count} missing values")
    
df.printSchema()

# Extract all unique authors
all_authors = df.select("authors_list").rdd.flatMap(lambda x: x[0]).distinct().collect()

# Apply the MultiHot Encoding UDF
print(len(all_authors))


# Print unique publishers
unique_publishers = df.select("publisher").distinct().rdd.flatMap(lambda x: x).collect()
print(len(unique_publishers))

columns_to_drop = ['authors_list', 'authors']
df = df.drop(*columns_to_drop)

df = df.withColumn("text", coalesce(col("Title"), col("description")))


df = df.drop("description")


# Assuming df is your DataFrame
df = df.withColumn("publishedYear", year("publishedDate"))

# Alternatively, you can calculate the age
df = df.withColumn("age", year(current_date()) - year("publishedDate")) 

columns_to_drop = ['publishedDate', 'publishedYear']
df = df.drop(*columns_to_drop)

df = df.withColumn("Impact", col("Impact").cast("double"))




/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Setting default log level to "

root
 |-- _c0: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- publishedDate: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- Impact: string (nullable = true)

Column 'Index': 0 missing values
Column 'Title': 0 missing values
Column 'description': 12749 missing values
Column 'authors': 2723 missing values
Column 'publisher': 0 missing values
Column 'publishedDate': 393 missing values
Column 'categories': 0 missing values
Column 'Impact': 0 missing values
root
 |-- Index: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- publishedDate: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- Impact: string (nullable = true)

Column 'Index': 0 missing values
Column 'Title'

In [2]:
embeddings_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def embed_sentences(text):
    embeddings = embeddings_model.encode([text])
    return embeddings.tolist()[0]

# Register the UDF
spark.udf.register("embed_sentences", embed_sentences, "array<double>")

# Apply the UDF to create a new column with embeddings
df = df.withColumn("text_embeddings", expr("embed_sentences(text)"))

df = df.drop("description")


df.show(2)

+-----+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+---+--------------------+
|Index|               Title|           publisher|          categories|           Impact|     categories_list|                text|age|     text_embeddings|
+-----+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+---+--------------------+
|    0|Its Only Art If I...|Smithsonian Insti...|['Comics & Graphi...|784.3039243054303|[Comics & Graphic...|Its Only Art If I...| 28|[-0.0155602190643...|
|    1|Dr. Seuss: Americ...|           A&C Black|['Biography & Aut...|825.4655354138016|[Biography & Auto...|Dr. Seuss: Americ...| 19|[0.21532303094863...|
+-----+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+---+--------------------+
only showing top 2 rows



In [3]:
embedding_length= 384

# Create separate columns for each dimension of the text_embeddings
for i in range(embedding_length):
        col_name = f"text_embeddings{i + 1}"
        df = df.withColumn(col_name, df["text_embeddings"][i])

In [36]:
all_categories = df.select("categories_list").rdd.flatMap(lambda x: x[0]).distinct().collect()


In [40]:
from pyspark.sql.functions import col, when, array_contains

for category in all_categories:
    # Use the 'when' function to create a binary column
    df = df.withColumn(f"category_{category}", when(array_contains(col("categories_list"), category), 1).otherwise(0))


In [42]:
df.show(2)

+-----+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+---+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+------------------+--------------------+-------------------+--------------------+-------------------+--------------------+------------------+--------------------+--------------------+--------------------+-------------------+-------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+-------------------+-------------------+-------------------+--------------------+--------------------

In [ ]:
columns_to_drop = ['categories_list', 'categories']
df = df.drop(*columns_to_drop)


In [47]:
from pyspark.sql.functions import size, split, length
df = df.withColumn("title_word_count", size(split(col("Title"), " ")))
df = df.withColumn("title_avg_word_length", length(col("Title")) / col("title_word_count"))
df = df.withColumn("char_count", length(col("Title")))
df = df.withColumn("contains_digits", when(col("Title").rlike("\\d"), 1).otherwise(0))
df = df.withColumn("contains_uppercase", when(col("Title").rlike("[A-Z]"), 1).otherwise(0))
df = df.withColumn("contains_punctuation", when(col("Title").rlike("[^\w\s]"), 1).otherwise(0))


In [53]:
from textblob import TextBlob
# Define a UDF for sentiment analysis using TextBlob
def analyze_sentiment(title):
    blob = TextBlob(title)
    return blob.sentiment.polarity

sentiment_udf = udf(analyze_sentiment, StringType())

# Apply the UDF to the 'Title' column
df = df.withColumn("sentiment_score", sentiment_udf(col("Title")))

In [54]:
df.select('sentiment_score').show(10)

+-------------------+
|    sentiment_score|
+-------------------+
|                0.0|
|                0.0|
|                0.5|
|                0.0|
|                0.0|
|                0.0|
|0.11818181818181818|
|0.03333333333333333|
|                0.0|
|                0.0|
+-------------------+
only showing top 10 rows



In [55]:
df.columns

['Index',
 'Title',
 'publisher',
 'categories',
 'Impact',
 'categories_list',
 'text',
 'age',
 'text_embeddings',
 'text_embeddings1',
 'text_embeddings2',
 'text_embeddings3',
 'text_embeddings4',
 'text_embeddings5',
 'text_embeddings6',
 'text_embeddings7',
 'text_embeddings8',
 'text_embeddings9',
 'text_embeddings10',
 'text_embeddings11',
 'text_embeddings12',
 'text_embeddings13',
 'text_embeddings14',
 'text_embeddings15',
 'text_embeddings16',
 'text_embeddings17',
 'text_embeddings18',
 'text_embeddings19',
 'text_embeddings20',
 'text_embeddings21',
 'text_embeddings22',
 'text_embeddings23',
 'text_embeddings24',
 'text_embeddings25',
 'text_embeddings26',
 'text_embeddings27',
 'text_embeddings28',
 'text_embeddings29',
 'text_embeddings30',
 'text_embeddings31',
 'text_embeddings32',
 'text_embeddings33',
 'text_embeddings34',
 'text_embeddings35',
 'text_embeddings36',
 'text_embeddings37',
 'text_embeddings38',
 'text_embeddings39',
 'text_embeddings40',
 'text_embed

In [87]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, expr, coalesce, current_date, year, udf
from pyspark.ml.linalg import Vectors, VectorUDT
from sentence_transformers import SentenceTransformer
from pyspark.sql.types import ArrayType, StringType
from textblob import TextBlob
from pyspark.sql.functions import size, split, length, array_contains


# Create a Spark session
spark = SparkSession.builder.appName("BookEmbeddings1").getOrCreate()

# Read data directly into DataFrame with appropriate CSV options
csv_path = "books_task.csv"
df = spark.read.option("header", "true").option("quote", "\"").option("escape", "\"").csv(csv_path)

df.printSchema()

# Rename the '_c0' column to 'Index'
df = df.withColumnRenamed("_c0", "Index")

# Convert 'publishedDate' to date format
df = df.withColumn("publishedDate", col("publishedDate").cast("date"))

# Count missing values in each column
missing_counts = [df.where(col(c).isNull()).count() for c in df.columns]

# Create a dictionary to map column names to missing value counts
missing_dict = dict(zip(df.columns, missing_counts))

# Display the missing value counts for each column
for column, missing_count in missing_dict.items():
    print(f"Column '{column}': {missing_count} missing values")
    
# Replace missing values with a default date 
default_date = "1900-01-01"
df = df.withColumn("publishedDate", when(col("publishedDate").isNull(), default_date).otherwise(col("publishedDate")))

df.printSchema()

# Count missing values in each column
missing_counts = [df.where(col(c).isNull()).count() for c in df.columns]

# Create a dictionary to map column names to missing value counts
missing_dict = dict(zip(df.columns, missing_counts))

# Display the missing value counts for each column
for column, missing_count in missing_dict.items():
    print(f"Column '{column}': {missing_count} missing values")
    

# Define a UDF to convert string representation of list to actual list
@udf(ArrayType(StringType()))
def convert_to_list(authors_str):
    return (authors_str[1:-1].replace("'", "").split(',')) if authors_str is not None else []

# Apply the UDF to the 'authors' column
df = df.withColumn("authors_list", convert_to_list(col("authors")))

# Apply the UDF to the 'categories' column
df = df.withColumn("categories_list", convert_to_list(col("categories")))


#Count missing values in each column
missing_counts = [df.where(col(c).isNull()).count() for c in df.columns]

# Create a dictionary to map column names to missing value counts
missing_dict = dict(zip(df.columns, missing_counts))

# Display the missing value counts for each column
for column, missing_count in missing_dict.items():
    print(f"Column '{column}': {missing_count} missing values")
    
df.printSchema()

# Extract all unique authors
all_authors = df.select("authors_list").rdd.flatMap(lambda x: x[0]).distinct().collect()

# Apply the MultiHot Encoding UDF
print(len(all_authors))


# Print unique publishers
unique_publishers = df.select("publisher").distinct().rdd.flatMap(lambda x: x).collect()
print(len(unique_publishers))


df = df.withColumn("text", coalesce(col("Title"), col("description")))


# Assuming df is your DataFrame
df = df.withColumn("publishedYear", year("publishedDate"))

# calculate the age
df = df.withColumn("age", year(current_date()) - year("publishedDate")) 


df = df.withColumn("Impact", col("Impact").cast("double"))




embeddings_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def embed_sentences(text):
    embeddings = embeddings_model.encode([text])
    return embeddings.tolist()[0]

# Register the UDF
spark.udf.register("embed_sentences", embed_sentences, "array<double>")

# Apply the UDF to create a new column with embeddings
df = df.withColumn("text_embeddings", expr("embed_sentences(text)"))


embedding_length= 384

# Create separate columns for each dimension of the text_embeddings
for i in range(embedding_length):
        col_name = f"text_embeddings{i + 1}"
        df = df.withColumn(col_name, df["text_embeddings"][i])

all_categories = df.select("categories_list").rdd.flatMap(lambda x: x[0]).distinct().collect()



for category in all_categories:
    # Use the 'when' function to create a binary column
    df = df.withColumn(f"category_{category}", when(array_contains(col("categories_list"), category), 1).otherwise(0))

df.show(2)


df = df.withColumn("title_word_count", size(split(col("Title"), " ")))
df = df.withColumn("title_avg_word_length", length(col("Title")) / col("title_word_count"))
df = df.withColumn("char_count", length(col("Title")))
df = df.withColumn("contains_digits", when(col("Title").rlike("\\d"), 1).otherwise(0))
df = df.withColumn("contains_uppercase", when(col("Title").rlike("[A-Z]"), 1).otherwise(0))
df = df.withColumn("contains_punctuation", when(col("Title").rlike("[^\w\s]"), 1).otherwise(0))


# Define a UDF for sentiment analysis using TextBlob
def analyze_sentiment(title):
    blob = TextBlob(title)
    return blob.sentiment.polarity

sentiment_udf = udf(analyze_sentiment, StringType())

# Apply the UDF to the 'Title' column
df = df.withColumn("sentiment_score", sentiment_udf(col("Title")))

df = df.withColumn("sentiment_score", col("sentiment_score").cast("double"))


'''columns_to_drop = ['description','categories_list', 'categories','publishedDate', 'publishedYear','authors_list', 'authors','Title','text','text_embeddings']
df = df.drop(*columns_to_drop)
'''

root
 |-- _c0: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- publishedDate: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- Impact: string (nullable = true)

Column 'Index': 0 missing values
Column 'Title': 0 missing values
Column 'description': 12749 missing values
Column 'authors': 2723 missing values
Column 'publisher': 0 missing values
Column 'publishedDate': 393 missing values
Column 'categories': 0 missing values
Column 'Impact': 0 missing values
root
 |-- Index: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- publishedDate: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- Impact: string (nullable = true)

Column 'Index': 0 missing values
Column 'Title'

In [88]:
columns_to_drop = ['description','categories_list', 'categories','publishedDate', 'publishedYear','authors_list', 'authors','Title','text','text_embeddings']
df = df.drop(*columns_to_drop)

In [90]:
df.write.parquet("final_highlevel_data1")

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 45352)
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/dineshreddythimmasani/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/dineshreddythimmasani/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/dineshreddythimmasani/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
Traceback (most recent call last):
  File "/usr/lib/python3.10/socketser

Py4JError: An error occurred while calling o7554.parquet

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/dineshreddythimmasani/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/dineshreddythimmasani/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/dineshreddythimmasani/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


In [68]:
df.columns

['Index',
 'publisher',
 'Impact',
 'age',
 'text_embeddings1',
 'text_embeddings2',
 'text_embeddings3',
 'text_embeddings4',
 'text_embeddings5',
 'text_embeddings6',
 'text_embeddings7',
 'text_embeddings8',
 'text_embeddings9',
 'text_embeddings10',
 'text_embeddings11',
 'text_embeddings12',
 'text_embeddings13',
 'text_embeddings14',
 'text_embeddings15',
 'text_embeddings16',
 'text_embeddings17',
 'text_embeddings18',
 'text_embeddings19',
 'text_embeddings20',
 'text_embeddings21',
 'text_embeddings22',
 'text_embeddings23',
 'text_embeddings24',
 'text_embeddings25',
 'text_embeddings26',
 'text_embeddings27',
 'text_embeddings28',
 'text_embeddings29',
 'text_embeddings30',
 'text_embeddings31',
 'text_embeddings32',
 'text_embeddings33',
 'text_embeddings34',
 'text_embeddings35',
 'text_embeddings36',
 'text_embeddings37',
 'text_embeddings38',
 'text_embeddings39',
 'text_embeddings40',
 'text_embeddings41',
 'text_embeddings42',
 'text_embeddings43',
 'text_embeddings44'

In [73]:
df1 = df.withColumn("sentiment_score", col("sentiment_score").cast("double"))


In [69]:
from pyspark.ml.feature import PCA


In [70]:
non_pca_features = ['Index','publisher','Impact']
pca_features = [item for item in df.columns if item not in non_pca_features]

In [74]:
from pyspark.ml import Pipeline

# Assemble features for PCA
assembler = VectorAssembler(inputCols=pca_features, outputCol="features")

# Perform PCA
pca = PCA(k=50, inputCol="features", outputCol="pca_result")
model = pca.fit(df_assembled)
df_pca_result = model.transform(df_assembled)

# Show the DataFrame with PCA results
df_pca_result.select("Index","pca_result","publisher","Impact").show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|pca_

In [77]:
df_new = df_pca_result.select("Index","pca_result","publisher","Impact")

In [86]:
df_new = df_new.withColumn("pca_result_str", expr("CONCAT_WS(',', pca_result.type, pca_result.size)"))


AnalysisException: [INVALID_EXTRACT_BASE_FIELD_TYPE] Can't extract a value from "pca_result". Need a complex type [STRUCT, ARRAY, MAP] but got "STRUCT<type: TINYINT, size: INT, indices: ARRAY<INT>, values: ARRAY<DOUBLE>>".; line 1 pos 15

In [ ]:
df_new = df_pca_result.select("Index","pca_result_str","publisher","Impact")

In [84]:
df_new.write.parquet("final_output_test1")

In [85]:
# Specify the path where you want to save the CSV file
output_csv_path = "test.csv"

# Write the DataFrame to a CSV file
df_new.write.csv(output_csv_path, header=True)

AnalysisException: [UNSUPPORTED_DATA_TYPE_FOR_DATASOURCE] The CSV datasource doesn't support the column `pca_result` of the type "STRUCT<type: TINYINT, size: INT, indices: ARRAY<INT>, values: ARRAY<DOUBLE>>".

In [94]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, expr, coalesce, current_date, year, udf
from pyspark.ml.linalg import Vectors, VectorUDT
from sentence_transformers import SentenceTransformer
from pyspark.sql.types import ArrayType, StringType




# Create a Spark session
spark = SparkSession.builder.appName("BookEmbeddings1").getOrCreate()

# Read data directly into DataFrame with appropriate CSV options
csv_path = "books_task.csv"
df = spark.read.option("header", "true").option("quote", "\"").option("escape", "\"").csv(csv_path)

df.printSchema()

# Rename the '_c0' column to 'Index'
df = df.withColumnRenamed("_c0", "Index")

# Convert 'publishedDate' to date format
df = df.withColumn("publishedDate", col("publishedDate").cast("date"))


ConnectionRefusedError: [Errno 111] Connection refused